In [25]:
import re
def loadIntoPatternsStore(patternsStore, line):
    components = line.split("\t")
    groupId = components[0]
    websiteName = components[1]
    patternType = components[2]
    missed = int(components[3])
    junk = int(components[4])
    total = int(components[5])
    leftPattern = components[6]
    rightPattern = components[7]

    if groupId in patternsStore:
        (websiteName, patternType, missed, junk, total, patterns) = patternsStore[groupId]
        patterns.append((leftPattern, rightPattern))
    else:
        patternsStore[groupId] = (websiteName, patternType, missed, junk, total, [(leftPattern, rightPattern)])
def readProductPatternStore(fileLocation):
    patternsStore = {}
    index=0
    with open(fileLocation) as f:
        for line in f: 
            if index==0:
                index=1
                continue
            line = line.strip() #or some other preprocessing
            loadIntoPatternsStore(patternsStore, line)
#             print("line is " + line)
    output = []
    for (k, v) in patternsStore.items():
        output.append(v)
    return output
# patternsStore = readProductPatternStore(productPatternsLocation)

def mergePatternsGroup(p1, p2, p3):
    p1.extend(p2)
    p1.extend(p3)
    return p1
            
def writePatternsToFile(fileLocation, productPatterns):
    header = "GroupId\tWebsiteName\tPatternType\tMissed\tJunk\tOutof\tLeftPattern\tRightPattern"
    groupId = 1
    output = []
    output.append(header)
    for (website, pType, missed, junk, total, patterns) in productPatterns:
        rowPrefix = str(groupId) + "\t" + website + "\t" + pType
        rowPrefix+="\t" + str(missed) + "\t" + str(junk) + "\t" + str(total)
        for (l, r) in patterns:
            row = rowPrefix + "\t" + l + "\t" + r
            output.append(row)
        groupId+=1
#     print("output is ")
#     for item in output:
#         print(item)
    writeListToFile(fileLocation, output)
    print("Output written at location: " + fileLocation)
    
    
def writeListToFile(loc, l):
    with open(loc, 'w') as f:
        for item in l:
            f.write(item+"\n")


In [26]:
print("Enter patterns Location: ")
patternsLocation = raw_input()
productNameLoc     = patternsLocation + "/ProductPatterns.tsv"
categoriesLoc      = patternsLocation + "/categoryPatterns.tsv"
firstCategoriesLoc = patternsLocation + "/firstCategories.tsv"
lastCategoriesLoc  = patternsLocation + "/lastCategories.tsv"
specLoc            = patternsLocation + "/specifications.tsv"
firstSpecLoc       = patternsLocation + "/firstSpecifications.tsv"
lastSpecLoc        = patternsLocation + "/lastSpecifications.tsv"
productNamePatternsGroup = readProductPatternStore(productNameLoc)
categoriesPatternsGroup  = readProductPatternStore(categoriesLoc)
catFirstPatternsGroup    = readProductPatternStore(firstCategoriesLoc)
catLastPatternsGroup     = readProductPatternStore(lastCategoriesLoc)
specLocPatternsGroup     = readProductPatternStore(specLoc)
specFirstPatternsGroup   = readProductPatternStore(firstSpecLoc)
specLastPatternsGroup    = readProductPatternStore(lastSpecLoc)
mergedCategoryGroup      = mergePatternsGroup(categoriesPatternsGroup, catFirstPatternsGroup, catLastPatternsGroup)
mergedSpecGroup          = mergePatternsGroup(specLocPatternsGroup, specFirstPatternsGroup, specLastPatternsGroup)
# writePatternsToFile(patternsLocation+"/mergedCategories.tsv", mergedCategoryGroup)
# writePatternsToFile(patternsLocation+"/mergedSpec.tsv", mergedSpecGroup)



Enter patterns Location: 
patternsLearnt


In [31]:

def doProcessingWithoutClass(s):
    i = "class[\s]*=[\s]*"
    o       = "class="
    s = re.sub(i, o, s)
    i = "([^class])=\"[^\"]*\""
    o = "\\1=\"\""
    return re.sub(i, o, s)

#read the page from pageLocation
def readPage(pageLocation):
    htmlPageContent = ""
    with open(pageLocation, 'r') as myfile:
        htmlPageContent = myfile.read().strip()
    return htmlPageContent


#remove elements which were actually tag
def isTag(s):
    if s.find("<")!=-1 and s.find(">")!=-1:
        return True
    return False


#preprocess results before writing it to file
def preprocessResults(output):
    result = []
    for o in output:
        if isTag(o):
            continue
        result.append(o)
    return result

#document processing logic at this stage is about removing multiple whitespaces into single one 
def preprocessDocument(document):
    return ' '.join(document.split())



#Pattern is (l, r) and match them to htmlPageContent
def findEntitySetwrtPattern(htmlPageContent, (l, r)):
    #for each start location of pattern find its end
    #for each end page find the pattern right
    #extract everything till that point
    #after extraction move one point above that pattern string
    results = []
    for m in re.finditer(re.escape(l), htmlPageContent):
        start = m.start()
        end = m.end()
        rightPage = htmlPageContent[end:]
        rightLoc  = rightPage.find(r)
        if rightLoc==-1:
            break
        element = rightPage[:rightLoc]
        if len(element)>1 and len(element)<300:
            results.append(element)
    return set(results)


def extractSet(patterns, htmlPageContent):
    output = []
    for pattern in patterns:
        output.extend(findEntitySetwrtPattern(htmlPageContent, pattern))
    return set(output)

def applyPatternsToPageLocation(patternsStore, inputPageLocation):
#     patternsStore      = readProductPatternStore(patternsFileLocation)
    plainPageContent   = preprocessDocument(readPage(inputPageLocation))
    noValuePageContent = doProcessingWithoutClass(preprocessDocument(readPage(inputPageLocation)))
    results = []
    for pattern in patternsStore:
        (websiteName, patternType, missed, junk, total, lrPatterns) = pattern
        if patternType=="plainPattern":
            pageContent = plainPageContent
        else:
            pageContent = noValuePageContent
        resultsPerPattern = preprocessResults(list(extractSet(lrPatterns, pageContent)))
        if (len(resultsPerPattern)>0):
            results.extend(resultsPerPattern)
#             results.append((websiteName, resultsPerPattern, missed, junk, total))
#         print(resultsPerPattern)
    return list(set(results))

def preprocessCategories(categories, productNames):
    if "Home" in categories:
        categories.remove("Home")
    if len(productNames)==1:
        if productNames[0] in categories:
            categories.remove(productNames[0])
    return categories


print("Enter the input page lcoation")
inputPageLocation = raw_input()
productNames = applyPatternsToPageLocation(productNamePatternsGroup, inputPageLocation)
categories   = applyPatternsToPageLocation(mergedCategoryGroup, inputPageLocation)
categories   = preprocessCategories(categories, list(productNames))
specs        = applyPatternsToPageLocation(mergedSpecGroup, inputPageLocation)
print("Product names: ")
print(productNames)
print("Categories: ")
print(categories)
print("Specs: ")
print(specs)

Enter the input page lcoation
unseen/page.html
Product names: 
['Grand Theft Auto IV (for PS3)']
Categories: 
['Gaming', 'PS3 Games', 'Games', 'PS3 Physical Games']
Specs: 
['Game Modes: Multi-Player, Single-Player', 'Genre: Action-Adventure', 'Edition: Standard Edition', 'Platform: PS3']


In [ ]:
import re
import cgi
def loadIntoPatternsStore(patternsStore, line):
    components = line.split("\t")
    groupId = components[0]
    websiteName = components[1]
    patternType = components[2]
    missed = int(components[3])
    junk = int(components[4])
    total = int(components[5])
    leftPattern = components[6]
    rightPattern = components[7]

    if groupId in patternsStore:
        (websiteName, patternType, missed, junk, total, patterns) = patternsStore[groupId]
        patterns.append((leftPattern, rightPattern))
    else:
        patternsStore[groupId] = (websiteName, patternType, missed, junk, total, [(leftPattern, rightPattern)])
def readProductPatternStore(fileLocation):
    patternsStore = {}
    index=0
    with open(fileLocation) as f:
        for line in f: 
            if index==0:
                index=1
                continue
            line = line.strip() #or some other preprocessing
            loadIntoPatternsStore(patternsStore, line)
#             print("line is " + line)
    output = []
    for (k, v) in patternsStore.items():
        output.append(v)
    return output
# patternsStore = readProductPatternStore(productPatternsLocation)

def mergePatternsGroup(p1, p2, p3):
    p1.extend(p2)
    p1.extend(p3)
    return p1
            
def writePatternsToFile(fileLocation, productPatterns):
    header = "GroupId\tWebsiteName\tPatternType\tMissed\tJunk\tOutof\tLeftPattern\tRightPattern"
    groupId = 1
    output = []
    output.append(header)
    for (website, pType, missed, junk, total, patterns) in productPatterns:
        rowPrefix = str(groupId) + "\t" + website + "\t" + pType
        rowPrefix+="\t" + str(missed) + "\t" + str(junk) + "\t" + str(total)
        for (l, r) in patterns:
            row = rowPrefix + "\t" + l + "\t" + r
            output.append(row)
        groupId+=1
#     print("output is ")
#     for item in output:
#         print(item)
    writeListToFile(fileLocation, output)
    print("Output written at location: " + fileLocation)
    
    
def writeListToFile(loc, l):
    with open(loc, 'w') as f:
        for item in l:
            f.write(item+"\n")

def doProcessingWithoutClass(s):
    i = "class[\s]*=[\s]*"
    o       = "class="
    s = re.sub(i, o, s)
    i = "([^class])=\"[^\"]*\""
    o = "\\1=\"\""
    return re.sub(i, o, s)

#read the page from pageLocation
def readPage(pageLocation):
    htmlPageContent = ""
    with open(pageLocation, 'r') as myfile:
        htmlPageContent = myfile.read().strip()
    return htmlPageContent


#remove elements which were actually tag
def isTag(s):
    if s.find("<")!=-1 and s.find(">")!=-1:
        return True
    return False


#preprocess results before writing it to file
def preprocessResults(output):
    result = []
    for o in output:
        if isTag(o):
            continue
        result.append(o)
    return result

#document processing logic at this stage is about removing multiple whitespaces into single one 
def preprocessDocument(document):
    return ' '.join(document.split())



#Pattern is (l, r) and match them to htmlPageContent
def findEntitySetwrtPattern(htmlPageContent, (l, r)):
    #for each start location of pattern find its end
    #for each end page find the pattern right
    #extract everything till that point
    #after extraction move one point above that pattern string
    results = []
    for m in re.finditer(re.escape(l), htmlPageContent):
        start = m.start()
        end = m.end()
        rightPage = htmlPageContent[end:]
        rightLoc  = rightPage.find(r)
        if rightLoc==-1:
            break
        element = rightPage[:rightLoc]
        if len(element)>1 and len(element)<300:
            results.append(element)
    return set(results)


def extractSet(patterns, htmlPageContent):
    output = []
    for pattern in patterns:
        output.extend(findEntitySetwrtPattern(htmlPageContent, pattern))
    return set(output)

def applyPatternsToPageLocation(patternsStore, inputPageLocation):
#     patternsStore      = readProductPatternStore(patternsFileLocation)
    plainPageContent   = preprocessDocument(readPage(inputPageLocation))
    noValuePageContent = doProcessingWithoutClass(preprocessDocument(readPage(inputPageLocation)))
    results = []
    for pattern in patternsStore:
        (websiteName, patternType, missed, junk, total, lrPatterns) = pattern
        if patternType=="plainPattern":
            pageContent = plainPageContent
        else:
            pageContent = noValuePageContent
        resultsPerPattern = preprocessResults(list(extractSet(lrPatterns, pageContent)))
        if (len(resultsPerPattern)>0):
            results.extend(resultsPerPattern)
#             results.append((websiteName, resultsPerPattern, missed, junk, total))
#         print(resultsPerPattern)
    return list(set(results))

def preprocessCategories(categories, productNames):
    if "Home" in categories:
        categories.remove("Home")
    if len(productNames)==1:
        if productNames[0] in categories:
            categories.remove(productNames[0])
    return categories


patternsLocation = "patterns"
productNameLoc     = patternsLocation + "/ProductPatterns.tsv"
categoriesLoc      = patternsLocation + "/categoryPatterns.tsv"
firstCategoriesLoc = patternsLocation + "/firstCategories.tsv"
lastCategoriesLoc  = patternsLocation + "/lastCategories.tsv"
specLoc            = patternsLocation + "/specifications.tsv"
firstSpecLoc       = patternsLocation + "/firstSpecifications.tsv"
lastSpecLoc        = patternsLocation + "/lastSpecifications.tsv"
productNamePatternsGroup = readProductPatternStore(productNameLoc)
categoriesPatternsGroup  = readProductPatternStore(categoriesLoc)
catFirstPatternsGroup    = readProductPatternStore(firstCategoriesLoc)
catLastPatternsGroup     = readProductPatternStore(lastCategoriesLoc)
specLocPatternsGroup     = readProductPatternStore(specLoc)
specFirstPatternsGroup   = readProductPatternStore(firstSpecLoc)
specLastPatternsGroup    = readProductPatternStore(lastSpecLoc)
mergedCategoryGroup      = mergePatternsGroup(categoriesPatternsGroup, catFirstPatternsGroup, catLastPatternsGroup)
mergedSpecGroup          = mergePatternsGroup(specLocPatternsGroup, specFirstPatternsGroup, specLastPatternsGroup)
# writePatternsToFile(patternsLocation+"/mergedCategories.tsv", mergedCategoryGroup)
# writePatternsToFile(patternsLocation+"/mergedSpec.tsv", mergedSpecGroup)
form = cgi.FieldStorage()
pageInputLocation =  form.getvalue('productPageInput')
print("Page input location is " + str(pageInputLocation))
# print("Enter the input page lcoation")
# inputPageLocation = raw_input()
productNames = applyPatternsToPageLocation(productNamePatternsGroup, inputPageLocation)
categories   = applyPatternsToPageLocation(mergedCategoryGroup, inputPageLocation)
categories   = preprocessCategories(categories, list(productNames))
specs        = applyPatternsToPageLocation(mergedSpecGroup, inputPageLocation)
print("Product names: ")
print(productNames)
print("Categories: ")
print(categories)
print("Specs: ")
print(specs)

